In [ ]:
import pandas as pd
import numpy as np
import os
from smart import get_smart_profiles, get_Tz
from dumb import get_dumb_profiles
from random import choice
from time import time

In [ ]:
def get_production_consumption(enddatetime = '2017-12-31 23:00:00'):
    startdatetime='2017-01-01 00:00:00'
    df = pd.read_csv(os.path.join('data','Productie en verbruik info Core.csv'), delimiter=';')
    df.Datum = pd.to_datetime(df.Datum + ' ' + df.Tijd)
    df.rename(columns={'Datum':'timestamp'}, inplace=True)
    df.drop(['Tijd'], axis = 1, inplace = True)
    df.set_index('timestamp', inplace=True)
    df = df.asfreq('15T')
    df.interpolate(inplace=True)

    return df.loc[startdatetime:enddatetime]

In [ ]:
def get_availability_profiles(df):

    df_av = pd.read_excel(os.path.join('data','Laadprofielen.xlsx'), sheet_name='3. State of Charge', header=None)
    # Select rows 4 and onwards and columns D through AA
    df_av = df_av.iloc[3:, 3:42]
    # Reset column names
    df_av.columns = df_av.iloc[0]
    df_av = df_av[1:]

    for column in df_av.columns:
        col_year = list(df_av[column])*52 + list(df_av[column])[:96]
        df[column] = col_year[:len(df)]

    return df

In [ ]:
def get_prices(df, dynamic_prices, capaciteitspiek, nb_users):
    df_prices = pd.read_csv(os.path.join('data','BelpexFilter.csv'), delimiter=';')
    df_prices.rename(columns={'Date':'timestamps'}, inplace=True)
    df_prices.timestamps = pd.to_datetime(df_prices.timestamps)
    df_prices.sort_values('timestamps', inplace=True)
    df_prices.set_index('timestamps', inplace=True)
    df_prices = df_prices.asfreq('H')
    df_prices = df_prices.resample('15T').interpolate()    
    df_prices.energy_price = (df_prices.energy_price*1e-3 + 0.204*1e-3)*4.3 + 40.4*capaciteitspiek/(365*96*nb_users) 
    df = pd.concat([df, df_prices.loc[df.index[0]:df.index[-1]]], axis=1)
    if dynamic_prices == False:
        df.energy_price = np.mean(df_prices.energy_price)
        print("#############################################################################",'\n',df.energy_price)

    return df

In [ ]:
def get_demandprof(user, df):
    """..."""
    demand = []
    Tz = get_Tz([user])[0]
    user['Tz'] = Tz
    for tz in Tz:
        demand.append((df[user.get('rand_profile')+' SOC [kWh]'].iloc[tz[0]], df[user.get('rand_profile')+' SOC [kWh]'].iloc[tz[1]-1]))
    return demand